# Get manuscript 'timeline'

## Description
Given an OpenReview invitation, list the events (submissions, revisions, reviews, comments, etc.) associated with each submission, arranged chronologically.

## Notes
* I am not sure what the difference is between `cdate`, `tcdate` and other timestamps. I will check with OpenReview folks to confirm that I am using the correct field.
* The list of events does not include comment edits, including score changes. I do not think these are available, but I can double check with OpenReview folks on this too.
* I haven't done any of the work to print this into a nice csv or anything, but I think it should be straightforward. You can use `event.stringify(pretty=False)` if you would prefer raw timestamps instead of human readable.

In [ ]:
import collections
from datetime import datetime
import openreview
import os

# Change these values according to your needs
INVITATION = 'ICLR.cc/2019/Conference/-/Blind_Submission'
LIMIT = 10 # Number of papers to download all revisions for

# A client is required for any OpenReview API actions
guest_client = openreview.Client(baseurl='https://api.openreview.net')

In [ ]:
class EventType(object):
    SUBMISSION = "Submission"
    REVISION = "Revision"
    REVIEW = "Review"
    COMMENT = "Comment"
    METAREVIEW = "Metareview"
    
GENERIC_INITIATOR_AUTHORS = "Authors"

def get_initiator(note):
    return note.signatures[0].split("/")[-1]

def clean_timestamp(timestamp):
    return datetime.fromtimestamp(
        timestamp/1000).strftime('%Y-%m-%dT%H:%M:%S') # Something human-readable for the file name

class Event(object):
    def __init__(self, forum, event_type, initiator, identifier, timestamp, reply_to):
        self.forum = forum
        self.event_type = event_type
        self.initiator = initiator
        self.identifier = identifier
        self.timestamp = timestamp
        self.reply_to = reply_to
    
    def stringify(self, pretty=True):
        if pretty:
            timestamp = clean_timestamp(self.timestamp)
        else:
            timestamp = self.timestamp
        return "\t".join(str(i) for i in [
            timestamp, self.forum, self.event_type,
            self.initiator, self.identifier, self.reply_to])

def get_events_from_forum(forum_note):
    events = []
    for revision in guest_client.get_references(
            referent=forum_note.id, original=True):
        events.append(Event(
            forum_note.id, EventType.REVISION, GENERIC_INITIATOR_AUTHORS,
            revision.id, revision.tcdate, revision.replyto))
  
    this_forum_notes = guest_client.get_notes(forum=forum_note.id)
    for note in this_forum_notes:
        initiator = get_initiator(note)
        
        if 'TL;DR' in note.content: # Find a better test for this!
            event_type = EventType.SUBMISSION
        elif 'review' in note.content:
            event_type = EventType.REVIEW
        elif 'metareview' in note.content:
            event_type = EventType.METAREVIEW
        else:
            event_type = EventType.COMMENT
        events.append(Event(
            forum_note.id, event_type, initiator,
            note.id, note.tcdate, note.replyto))
    return list(sorted(events, key=lambda x:x.timestamp))

In [ ]:
for i, forum_note in enumerate(openreview.tools.iterget_notes(
        guest_client, invitation=INVITATION)):
    
    print( "#" + forum_note.content["title"] + "\n")
    for event in get_events_from_forum(forum_note):
        print(event.stringify())
    print("\n")
    
    if i == LIMIT:
        break